In [5]:
import pandas as pd
import numpy as np

#### The data

In [6]:
test_df = pd.read_csv('../data/missingness/0/testing_set.csv')
test_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
2,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,1,0,0,0,1
3,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,1
4,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1


In [7]:
dqn_results_df = pd.read_csv('../test_dfs/negative_reward/step_reward_twelfth.csv')
dqn_results_df.head()

,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success
0,0.0,4.0,0.750000,1.0,1.0,"['ana', 'anti_dsdna_antibody', 'joint_involvem...",0.0,1.0
1,1.0,3.0,0.833333,1.0,1.0,"['ana', 'anti_dsdna_antibody', 'Lupus']",0.0,1.0
2,2.0,4.0,0.750000,1.0,1.0,"['ana', 'anti_dsdna_antibody', 'joint_involvem...",0.0,1.0
3,3.0,3.0,0.833333,1.0,1.0,"['ana', 'anti_dsdna_antibody', 'Lupus']",0.0,1.0
4,4.0,4.0,0.750000,1.0,1.0,"['ana', 'anti_dsdna_antibody', 'joint_involvem...",0.0,1.0


#### Some dictionaries

In [8]:
criteria_weights = {'fever':2, 'leukopenia':3, 'thrombocytopenia':4, 'auto_immune_hemolysis':4, 'delirium':2, 
                    'psychosis':3, 'seizure':5, 'non_scarring_alopecia':2, 'oral_ulcers':2, 
                    'subacute_cutaneous_lupus':4, 'discoid_lupus':4, 'acute_cutaneous_lupus':6, 
                    'pleural_effusion': 5, 'pericardial_effusion':5, 'acute_pericarditis':6, 'joint_involvement':6, 
                    'proteinuria':4, 'anti_cardioliphin_antibodies':2, 'anti_β2gp1_antibodies':2, 
                    'lupus_anti_coagulant':2, 'low_c3':3, 'low_c4':3, 'low_c3_and_low_c4':4, 'low_c3_or_low_c4':3, 
                    'anti_dsdna_antibody':6, 'anti_smith_antibody':6}

In [9]:
domains_feat_dict = {'constitutional': ['fever'],
                     'hematologic': ['leukopenia', 'thrombocytopenia', 'auto_immune_hemolysis'],
                     'neuropsychiatric': ['delirium', 'psychosis', 'seizure'],
                     'mucocutaneous': ['non_scarring_alopecia', 'oral_ulcers', 'cutaneous_lupus'],
                     'serosal': ['pleural_effusion', 'pericardial_effusion', 'acute_pericarditis'],
                     'musculoskeletal': ['joint_involvement'],
                     'renal': ['proteinuria'],
                     'antiphospholipid_antibodies': ['anti_cardioliphin_antibodies', 'anti_β2gp1_antibodies', 'lupus_anti_coagulant'],
                     'complement_proteins': ['low_c3', 'low_c4'],
                     'sle_specific_antibodies':['anti_dsdna_antibody', 'anti_smith_antibody']}

In [10]:
domains_max_scores_dict = {'constitutional': 2, 'hematologic': 4, 'neuropsychiatric': 5, 'mucocutaneous': 6,
                           'serosal': 6, 'musculoskeletal': 6, 'renal': 4, 'antiphospholipid_antibodies': 2, 
                           'complement_proteins': 4, 'sle_specific_antibodies':6}

#### Function to compute score

In [11]:
def compute_score(row):
    if row['ana'] == 0: # 0=negative, 1=positive 
        return 0
    total_row_score = 0
    for domain in domains_feat_dict.keys():
        domain_score = get_domain_score(row, domain)
        total_row_score += domain_score
    return total_row_score

In [25]:
def get_domain_score(row, domain):
    domain_features = domains_feat_dict[domain] # a list of features in that domain
    domain_score = 0
    if domain == 'complement_proteins':
        domain_score = get_c3_c4_score(row.low_c3, row.low_c4)
    else:
        for feat in domain_features:
            feat_score = get_feat_score(row, feat)
            if feat_score > domain_score:
                domain_score = feat_score
    if domain_score > domains_max_scores_dict[domain]:
        raise Exception('The score is too large for this domain!')
    return domain_score

In [22]:
def get_feat_score(row, feat):
    if feat == 'cutaneous_lupus':
        feat_score = get_cutaneous_lupus_score(row[feat])
    elif row[feat] <= 0:
        feat_score = 0
    else:
        feat_score = criteria_weights[feat]
    return feat_score

In [20]:
def get_c3_c4_score(c3, c4): # 0 - low, 1 is not low
    if (c3 == 1) & (c4 == 1):
        return criteria_weights['low_c3_and_low_c4']
    elif (c3 == 1) | (c4 == 1):
        return criteria_weights['low_c3_or_low_c4']
    else:
        return 0

In [21]:
def get_cutaneous_lupus_score(cutaneous_type):
    if cutaneous_type == 0: #negative for any form of cutaneous lupus
        return 0
    elif cutaneous_type == 1: #subacute cutaneous lupus
        return criteria_weights['subacute_cutaneous_lupus']
    elif cutaneous_type == 2: #acute cutaneous lupus
        return criteria_weights['acute_cutaneous_lupus']
    elif cutaneous_type == 3: #discoid lupus
        return criteria_weights['discoid_lupus']
    else:
        raise Exception('Unknown cutaneous lupus type!')

#### Testing the function

In [17]:
test_df_with_score = test_df.copy(deep=True)
test_df_with_score['score'] = test_df.apply(lambda row: compute_score(row), axis=1)
test_df_with_score.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label,score
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,17
1,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,12
2,1,0,0,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,1,19
3,1,1,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,1,12
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,16


In [18]:
test_df_with_score[test_df_with_score.label==0].score.max()

9

In [26]:
test_df_with_score['third_feat_score'] = test_df.apply(lambda row: compute_score(row), axis=1)
test_df_with_score.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label,score,new_feat_score,third_feat_score
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,17,17,17
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,12,12,12
2,1,0,0,0,0,0,1,0,0,0,...,0,1,1,0,0,0,1,19,19,19
3,1,1,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,1,12,12,12
4,1,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1,16,16,16
